In [ ]:
import scanpy as sc
from cellrank.kernels import VelocityKernel
from cellrank.estimators import GPCCA
import cellrank as cr
import scvelo as scv
import numpy as np
import pandas as pd
from pandas import DataFrame
import cloudpickle

In [2]:
def preparation(adata, model, x_key, add_spliced):
    adata.obsm['velocity'] = np.array(fm.transport(jnp.array(adata.obsm[x_key]), 
                                   condition=np.expand_dims(adata.obs.day.values, axis=1), 
                                   forward=True)
                      )-adata.obsm[x_key]
    if add_spliced:
        adata.layers['spliced'] = np.ones((adata.n_obs, adata.n_vars))
        adata.layers['unspliced'] = np.ones((adata.n_obs, adata.n_vars))

    scv.pp.moments(adata)
    return None

In [6]:
terminal_states = ['Amnion', 'Blood', 'Mid-Hind Gut', 'Foregut/Placodes', 'Mixed mesoderm', 'Extra-Embryonic ectoderm', 'Endothelial', 
                   'Cardiac', 'Endothelial/Mixed', 'Extra-Embryonic endoderm', 'Placodes/Extra-Embryonic mesoderm']
adata = sc.read_h5ad(f'/home/icb/jonas.flor/gastrulation_atlas/scvi/reference_query/scarches/1M/10k_genes/2/2048/invito/invito_adata.h5ad')
sc.pp.subsample(adata, n_obs=1000)
cluster_key = 'celltype'
n_states = int(len(adata.obs.celltype.unique())*1.0)
add_spliced=True
x_key = 'X_emb'
end = '_w'

In [7]:
with open(f'/home/icb/jonas.flor/gastrulation_atlas/scvi/reference_query/divergences/flow/1M/10k_genes/2/2048/integrated_cond_model{end}.pt', mode='rb') as file:
       fm = cloudpickle.load(file)

preparation(adata, fm, x_key, add_spliced)

Normalized count data: spliced, unspliced.
computing neighbors
    finished (0:00:11) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:03) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [9]:
vk = VelocityKernel(adata, attr='obsm', xkey=x_key, vkey="velocity").compute_transition_matrix()
g = GPCCA(vk)

g.compute_macrostates(cluster_key=cluster_key, n_states=n_states)
g.predict_terminal_states(n_states=len(terminal_states), method='top_n')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 5463.88cell/s]


GPCCA[kernel=VelocityKernel[n=1000], initial_states=None, terminal_states=['Blood_2', 'Blood_3', 'Blood_4', 'Blood_5', 'Extra-Embryonic endoderm_2', 'Extra-Embryonic mesoderm', 'Mixed mesoderm', 'Neural Crest/Mid-HindBrain_1', 'Neural Crest/Mid-HindBrain_3', 'Pharyngeal Mesoderm/Neural Crest']]

In [10]:
terminal_states_contained = 0
for state in terminal_states:
    if any([terminal_state.startswith(state) for terminal_state in g.terminal_states.cat.categories.astype('string')]):
        terminal_states_contained = terminal_states_contained+1
terminal_states_found = terminal_states_contained/len(terminal_states)

corrcet_terminal_states = 0
for terminal_state in g.terminal_states.cat.categories.astype('string'):
    if any([terminal_state.startswith(state) for state in terminal_states]):
        corrcet_terminal_states = corrcet_terminal_states+1
corrcet_terminal_states = corrcet_terminal_states/len(g.terminal_states.cat.categories)

In [11]:
terminal_state_anno = []
for state in g.macrostates.cat.categories.astype('string'):
    if any([terminal_state.startswith(state) for state in terminal_states]):
        terminal_state_anno.append(state)
terminal_state_anno

['Mixed mesoderm',
 'Neural Crest/Mid-HindBrain_1',
 'Extra-Embryonic endoderm_1',
 'Mixed',
 'Blood_1',
 'Blood_2',
 'Extra-Embryonic endoderm_2',
 'Blood_3',
 'Amnion',
 'Neural Crest/Mid-HindBrain_2',
 'Blood_4',
 'Blood_5',
 'Pharyngeal Mesoderm/Neural Crest',
 'Neural Crest/Mid-HindBrain_3',
 'Extra-Embryonic mesoderm']

In [12]:
g.macrostates.cat.categories.astype('string')

Index(['Mixed mesoderm', 'Neural Crest/Mid-HindBrain_1',
       'Extra-Embryonic endoderm_1', 'Mixed', 'Blood_1', 'Blood_2',
       'Extra-Embryonic endoderm_2', 'Blood_3', 'Amnion',
       'Neural Crest/Mid-HindBrain_2', 'Blood_4', 'Blood_5',
       'Pharyngeal Mesoderm/Neural Crest', 'Neural Crest/Mid-HindBrain_3',
       'Extra-Embryonic mesoderm'],
      dtype='string')

In [18]:
g.set_terminal_states(terminal_state_anno)

GPCCA[kernel=VelocityKernel[n=1000], initial_states=None, terminal_states=['Amnion', 'Blood_1', 'Blood_2', 'Blood_3', 'Blood_4', 'Blood_5', 'Extra-Embryonic endoderm_1', 'Extra-Embryonic endoderm_2', 'Extra-Embryonic mesoderm', 'Mixed', 'Mixed mesoderm', 'Neural Crest/Mid-HindBrain_1', 'Neural Crest/Mid-HindBrain_2', 'Neural Crest/Mid-HindBrain_3', 'Pharyngeal Mesoderm/Neural Crest']]

In [17]:
tmp = sc.read_h5ad('/home/icb/jonas.flor/gastrulation_atlas/scvi/reference_query/scarches/1M/10k_genes/2/2048/exvito/exvito_adata.h5ad', backed='r')
tmp.obs.celltype.unique()

['Blood', 'Extra-Embryonic mesoderm', 'Extra-Embryonic endoderm', 'Extra-Embryonic ectoderm', 'Pharyngeal Mesoderm/Neural Crest', ..., 'Cardiac', 'Foregut/Placodes', 'Mixed', 'Endothelial/Mixed', 'Placodes/Extra-Embryonic mesoderm']
Length: 15
Categories (15, object): ['Amnion', 'Blood', 'Cardiac', 'Endothelial', ..., 'Mixed mesoderm', 'Neural Crest/Mid-HindBrain', 'Pharyngeal Mesoderm/Neural Crest', 'Placodes/Extra-Embryonic mesoderm']

In [24]:
t = np.var(sc.read_h5ad('/home/icb/jonas.flor/gastrulation_atlas/scvi/reference_query/scarches/1M/10k_genes/2/2048/gastrulation/gastrulation_adata.h5ad', backed='r').obsm['X_emb'], axis=1)

In [25]:
np.min(t)

2.2796607

In [26]:
np.max(t)

16.442469

In [1]:
import scanpy as sc

In [ ]:
tmp = sc.read_h5ad('/lustre/groups/ml01/workspace/monge_velo/data/mouse_gastrulation_atlas/gastrulation_atlas_neighbors.h5ad', backed='r')

In [ ]:
tmp.X[:10,]